In [1]:
from phi import *
from intersect import *
from integrate import *

In [2]:
def k_fold_shift(f, k):
    if k <= 0:
        return f
    return shift(f) if k == 1 else shift(k_fold_shift(f, k-1))

In [3]:
PHI = {}

p = 3

In [4]:
generate_phi_m(PHI, 2, p=3)

In [7]:
for key in AAA:
    print(key, ': ', AAA[key], '\n', integrate(AAA[key], p), '\n')

1 :  {'blocks': [(['z', 1], 0), (['z', 2], 1)]} 
 x/2 + 1/2 

2 :  {'blocks': [([1, 'z', 1], 0), (['z', 1], 1), ([1, 'z', 2], 1), ([2], 1), (['z', 2], 2)]} 
 x**2/2 + x + 1/6 

